In [4]:
import pandas as pd 
from netCDF4 import Dataset
import glob

 ## Generating both Heat_Stress and humidity_DATA via following path 

In [3]:
# file=glob.glob("../../Humidity_DATA/HEAT_STRESS_nc_data/*.nc")
file=glob.glob("/global/cfs/projectdirs/m1532/Projects_MVP/MET_OFFICE_CLIMATE_DATA/HUMIDITY_data/*.nc")
file[0]


'/global/cfs/projectdirs/m1532/Projects_MVP/MET_OFFICE_CLIMATE_DATA/HUMIDITY_data/hadisd.3.3.0.202206p_19310101-20220701_043600-99999_humidity.nc'

In [16]:
nc = Dataset(file[0], 'r')
print(nc)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    title: HadISD
    institution: Met Office Hadley Centre, Exeter, UK
    source: HadISD data product
    references: Dunn, 2019, Met Office Hadley Centre Technical Note #103; Dunn, Willett, Parker & Mitchell, 2016, Geosci. Instrum. Method. Data Syst., 5, 473-491
    creator_name: Robert Dunn
    creator_url: www.metoffice.gov.uk
    creator_email: robert.dunn@metoffice.gov.uk
    keywords: sub-daily, station, extremes, temperature, dewpoint, pressure, wind_speed, wind_direction, cloud_cover
    summary: Quality-controlled, sub-daily, station dataset containing temperature, dewpoint, pressure, wind speed & direction and cloud cover.
    hadisd_version: 3.3.0.202206p
    isd_version: ---
    isd_date_full: 3 February 2022
    isd_date_update: 4 July 2022
    licence: HadISD is distributed under the Non Commercial Government Licence: http://www.nationalarchives.gov.uk/doc/non-commercial-government-lic

In [22]:
lat = nc.variables['latitude'][:][0]
lon = nc.variables['longitude'][:][0]
#checking files lat/lon
print(lat,lon)
#checking files variable and corresponding time
relative_humidity = nc.variables['relative_humidity'][:]
time = nc.variables['time'][:]
print(relative_humidity,time)

65.6 -37.633
[-- -- -- ... 79.09324645996094 79.1385498046875 77.08907318115234] [ 46740.  46764.  46812. ... 802053. 802054. 802055.]


### example of generating a county hourly different 

In [60]:
import pandas as pd
import numpy as np
import xarray as xr
import os

df_US=pd.read_csv("US_humidity_file.csv")
# df_US=pd.read_csv("US_heat_stress_file.csv")
df_US["lat"] = df_US["coordinate"].str.rsplit(',').str[0].str.strip('[').astype(float)
df_US["lon"] = df_US["coordinate"].str.rsplit(',').str[-1].str.strip(']').astype(float)
### Ingnore this since you are testing your imputation methods.
# county_humid=pd.read_csv("county_humid_data.csv").rename(columns={"closest_station_usaf_wban":"coordinate"}).drop(columns=["Unnamed: 0","X"])
# county_humid_file=county_humid.merge(df_US,how='left',on='coordinate')
# county_humid_file=county_humid_file[['county_x','Center_point','coordinate','km_to_closest_station','file_name']]


df_total = []
#should change 3 to the df length:
for i in range(len(df_US[0:3])):
    path = df_US[0:3]['file_name'].iloc[i]
    ds = xr.open_dataset(path)
    df = ds.to_dataframe().reset_index()
    print(df)
    #Change relative_humidity to any nc.variable to generate different files
    #dropna because there existed a lot NA values.
    relative_humidity=df[['time','longitude','latitude','relative_humidity','input_station_id','station_id']].dropna()
    #add county/corr info to the file
    relative_humidity['county'] = df_US[0:3]['county'].iloc[i]
    relative_humidity['coor'] = list(relative_humidity[['latitude', 'longitude']].to_records(index=False))  
    df_total.append(relative_humidity)
#     break

df_relative_humidity = pd.concat(df_total)
# df_relative_humidity.to_csv("apparent_temp/apparent_temp_0_200.csv",index=False)



        coordinate_length                time  longitude  latitude  elevation  \
0                       0 1997-01-23 17:00:00    -84.904    38.185      236.8   
1                       0 1997-01-31 08:00:00    -84.904    38.185      236.8   
2                       0 1997-01-31 10:00:00    -84.904    38.185      236.8   
3                       0 1997-01-31 12:00:00    -84.904    38.185      236.8   
4                       0 1997-01-31 13:00:00    -84.904    38.185      236.8   
...                   ...                 ...        ...       ...        ...   
219662                  0 2022-06-30 19:00:00    -84.904    38.185      236.8   
219663                  0 2022-06-30 20:00:00    -84.904    38.185      236.8   
219664                  0 2022-06-30 21:00:00    -84.904    38.185      236.8   
219665                  0 2022-06-30 22:00:00    -84.904    38.185      236.8   
219666                  0 2022-06-30 23:00:00    -84.904    38.185      236.8   

             station_id  te

### example output

In [61]:
df_relative_humidity

,time,longitude,latitude,relative_humidity,input_station_id,station_id,county,coor
6,1997-02-01 07:00:00,-84.904,38.185,56.454681,b'724233-99999',b'777777777777',Franklin County,"[38.185, -84.904]"
9,1997-02-03 15:00:00,-84.904,38.185,86.684135,b'724233-99999',b'777777777777',Franklin County,"[38.185, -84.904]"
18,1997-02-10 17:00:00,-84.904,38.185,77.754295,b'724233-99999',b'777777777777',Franklin County,"[38.185, -84.904]"
19,1997-02-10 18:00:00,-84.904,38.185,71.408524,b'724233-99999',b'777777777777',Franklin County,"[38.185, -84.904]"
20,1997-02-10 19:00:00,-84.904,38.185,71.587708,b'724233-99999',b'777777777777',Franklin County,"[38.185, -84.904]"
...,...,...,...,...,...,...,...,...
240870,2022-06-30 19:00:00,-96.180,41.763,44.661896,b'725527-94978',b'777777777777',Burt County,"[41.763, -96.18]"
240871,2022-06-30 20:00:00,-96.180,41.763,42.355873,b'725527-94978',b'777777777777',Burt County,"[41.763, -96.18]"
240872,2022-06-30 21:00:00,-96.180,41.763,39.814659,b'725527-94978',b'777777777777',Burt County,"[41.763, -96.18]"
240873,2022-06-30 22:00:00,-96.180,41.763,40.119972,b'725527-94978',b'777777777777',Burt County,"[41.763, -96.18]"
